In [1]:
import networkx as nx
import random
import matplotlib.pyplot as plt
import time
import math
import community as community_louvain
import networkx.algorithms.community as modularity
from networkx.algorithms.community.quality import modularity
from collections import defaultdict


G = nx.read_edgelist("C:/Users/gnssl/Downloads/facebook_combined.txt", 
                     create_using=nx.Graph(), nodetype=int)

partition_louvain = community_louvain.best_partition(G)
len_partition = len(partition_louvain)

population_size = 200
generations = 500
crossover_prob = 1.0
mutation_prob = 0.01
local_optimization_frequency=5

min_partition_size = 5
max_partition_size = 15

def initialize_population():
    population = []
    for idx in range(population_size):
        if idx%4==0:
            partition_louvain = community_louvain.best_partition(G)
            partition = [partition_louvain[node] for node in G.nodes]
            population.append(partition)
        else:
            partition = [random.randint(0, max_partition_size) for _ in range(len(G.nodes))]
            population.append(partition)
    return population

def compute_fitness(partition):
    communities = {key: value for key, value in zip(partition_louvain.keys(), partition)}
    new_dict = defaultdict(list)

    for key, value in communities.items():
        new_dict[value].append(key)

    result_dict = dict(new_dict)
    fitness = nx.community.modularity(G, result_dict.values())
    return fitness


def select_parents(population, fitness_values):
    parent1, parent2 = random.sample(population, 2)
    if compute_fitness(parent1) > compute_fitness(parent2):
        return parent1
    else:
        return parent2

def crossover(parent1, parent2):
    crossover_point = random.randint(1, len(parent1) - 1)
    child1 = parent1[:crossover_point] + parent2[crossover_point:]
    child2 = parent2[:crossover_point] + parent1[crossover_point:]
    return child1, child2

def mutate(individual, mutation_prob, max_partition_size):
    for i in range(len(individual)):
        if random.random() < mutation_prob:
            # Mutate the community assignment of the node to a random community
            individual[i] = random.randint(0, max_partition_size - 1)
    return individual

def partition_to_communities(partition):
    communities = {}
    for node, community_id in enumerate(partition):
        if community_id in communities:
            communities[community_id].add(node)
        else:
            communities[community_id] = {node}
    return list(communities.values())



def genetic_algorithm():
    population = initialize_population()
    max_fitness_values = []  # Store max fitness values for each generation

    for generation in range(generations):
        fitness_values = [compute_fitness(partition) for partition in population]
        max_fitness = max(fitness_values)
        max_fitness_values.append(max_fitness)
        avg_fitness = sum(fitness_values) / len(fitness_values)
        print('Generation', generation + 1, 'Max:', max_fitness)
        print('Generation', generation + 1, 'Avg:', avg_fitness)

        new_population = []
        for _ in range(population_size // 2):
            parent1 = select_parents(population, fitness_values)
            parent2 = select_parents(population, fitness_values)
            if random.random() < crossover_prob:
                child1, child2 = crossover(parent1, parent2)
                
                child1 = mutate(child1, mutation_prob, max_partition_size)
                child2 = mutate(child2, mutation_prob, max_partition_size)
                
                min_fitness_index = min(range(len(fitness_values)), key=fitness_values.__getitem__)
                population[min_fitness_index] = child1
                second_min_fitness_index = min(range(len(fitness_values)), key=lambda x: fitness_values[x] if x != min_fitness_index else float('inf'))
                population[second_min_fitness_index] = child2
                
        
        if generation % local_optimization_frequency == 0:
            random_idx = random.randint(1,population_size//2)
            for idx in range(0, random_idx, population_size//random_idx):
                partition_louvain = community_louvain.best_partition(G)
                partition = [partition_louvain[node] for node in G.nodes]
                population[idx] = partition
       
    # Select the best partition from the final population
    best_partition = max(population, key=compute_fitness)
    return best_partition, max_fitness_values

start = time.time()
best_partition, max_fitness_values = genetic_algorithm()

best_communities = partition_to_communities(best_partition)
end = time.time()
print('time:', end - start)

print("Best Partition:", best_communities)

Generation 1 Max: 0.8357057028579589
Generation 1 Avg: 0.20831973331551093
Generation 2 Max: 0.8357057028579589
Generation 2 Avg: 0.2120580025333394
Generation 3 Max: 0.8357057028579589
Generation 3 Avg: 0.21207171216683235
Generation 4 Max: 0.8357057028579589
Generation 4 Avg: 0.21209731620047187
Generation 5 Max: 0.8357057028579589
Generation 5 Avg: 0.21612657124397636
Generation 6 Max: 0.8357057028579589
Generation 6 Avg: 0.21613960455395953
Generation 7 Max: 0.8355621216480518
Generation 7 Avg: 0.29481859962561685
Generation 8 Max: 0.8355621216480518
Generation 8 Avg: 0.3022748336114099
Generation 9 Max: 0.8355621216480518
Generation 9 Avg: 0.30229200776100124
Generation 10 Max: 0.8355621216480518
Generation 10 Avg: 0.3062925091124848
Generation 11 Max: 0.8355621216480518
Generation 11 Avg: 0.3099844217934348
Generation 12 Max: 0.8355621216480518
Generation 12 Avg: 0.31796817364107044
Generation 13 Max: 0.8355621216480518
Generation 13 Avg: 0.31799247506597594
Generation 14 Max: 0.

Generation 109 Max: 0.8355621216480518
Generation 109 Avg: 0.8034795911036291
Generation 110 Max: 0.8355621216480518
Generation 110 Avg: 0.8043675374379238
Generation 111 Max: 0.8355621216480518
Generation 111 Avg: 0.8047648364873738
Generation 112 Max: 0.8357792944352957
Generation 112 Avg: 0.8053458387599778
Generation 113 Max: 0.8357792944352957
Generation 113 Avg: 0.8047179211301256
Generation 114 Max: 0.8357792944352957
Generation 114 Avg: 0.8054268765309149
Generation 115 Max: 0.8357792944352957
Generation 115 Avg: 0.805585440746951
Generation 116 Max: 0.8357792944352957
Generation 116 Avg: 0.8053446014392222
Generation 117 Max: 0.8357018122900586
Generation 117 Avg: 0.8063322146359367
Generation 118 Max: 0.8357018122900586
Generation 118 Avg: 0.8065943901017728
Generation 119 Max: 0.8357018122900586
Generation 119 Avg: 0.806360074667825
Generation 120 Max: 0.8357018122900586
Generation 120 Avg: 0.8065921606409249
Generation 121 Max: 0.8357018122900586
Generation 121 Avg: 0.80727

Generation 215 Max: 0.8356332777188367
Generation 215 Avg: 0.8206904432119516
Generation 216 Max: 0.8356332777188367
Generation 216 Avg: 0.8209119069950849
Generation 217 Max: 0.8357681525804703
Generation 217 Avg: 0.8211649924914652
Generation 218 Max: 0.8357681525804703
Generation 218 Avg: 0.8203279678986128
Generation 219 Max: 0.8357681525804703
Generation 219 Avg: 0.8212753493857015
Generation 220 Max: 0.8357681525804703
Generation 220 Avg: 0.8210498607960102
Generation 221 Max: 0.8357681525804703
Generation 221 Avg: 0.8208688965569348
Generation 222 Max: 0.8357681525804703
Generation 222 Avg: 0.8214764974110323
Generation 223 Max: 0.8357681525804703
Generation 223 Avg: 0.8213794782028009
Generation 224 Max: 0.8357681525804703
Generation 224 Avg: 0.8215290527346807
Generation 225 Max: 0.8357681525804703
Generation 225 Avg: 0.8216940922129509
Generation 226 Max: 0.8357681525804703
Generation 226 Avg: 0.8214648764999292
Generation 227 Max: 0.835686207502099
Generation 227 Avg: 0.8217

Generation 321 Max: 0.8355621216480518
Generation 321 Avg: 0.824649258122573
Generation 322 Max: 0.8357889403822414
Generation 322 Avg: 0.8247190729290979
Generation 323 Max: 0.8357889403822414
Generation 323 Avg: 0.8239622393601904
Generation 324 Max: 0.8357889403822414
Generation 324 Avg: 0.8246460666937964
Generation 325 Max: 0.8357889403822414
Generation 325 Avg: 0.8246423631317215
Generation 326 Max: 0.8357889403822414
Generation 326 Avg: 0.8246665769202651
Generation 327 Max: 0.8357889403822414
Generation 327 Avg: 0.8244476858915621
Generation 328 Max: 0.8357889403822414
Generation 328 Avg: 0.8240249576287373
Generation 329 Max: 0.8357889403822414
Generation 329 Avg: 0.823509383270866
Generation 330 Max: 0.8357889403822414
Generation 330 Avg: 0.8235323163459579
Generation 331 Max: 0.8357889403822414
Generation 331 Avg: 0.8237895769551404
Generation 332 Max: 0.8355621216480518
Generation 332 Avg: 0.8247742534647838
Generation 333 Max: 0.8355621216480518
Generation 333 Avg: 0.82474

Generation 427 Max: 0.8357430038334459
Generation 427 Avg: 0.8260638022862331
Generation 428 Max: 0.8357430038334459
Generation 428 Avg: 0.8247047966084246
Generation 429 Max: 0.8357430038334459
Generation 429 Avg: 0.8258400234340871
Generation 430 Max: 0.8357430038334459
Generation 430 Avg: 0.8258062278260466
Generation 431 Max: 0.8357430038334459
Generation 431 Avg: 0.8256307896114577
Generation 432 Max: 0.8357430038334459
Generation 432 Avg: 0.825953953316047
Generation 433 Max: 0.8357430038334459
Generation 433 Avg: 0.8251483176735093
Generation 434 Max: 0.8357430038334459
Generation 434 Avg: 0.8248697328859421
Generation 435 Max: 0.8357430038334459
Generation 435 Avg: 0.8261023139468123
Generation 436 Max: 0.8357430038334459
Generation 436 Avg: 0.825271399460084
Generation 437 Max: 0.8357430038334459
Generation 437 Avg: 0.8261590640133611
Generation 438 Max: 0.8357430038334459
Generation 438 Avg: 0.8261680094724771
Generation 439 Max: 0.8357430038334459
Generation 439 Avg: 0.82611